In [2]:
import json
import datetime
import time
import yaml
import pandas as pd
import mojito
import pprint
import requests
import pymongo

In [8]:
# 여기는 개인적인 key값이라 따로 key값을 적어놓고 불러오시는게 좋습니다.
with open('C:/Users/cc843/Desktop/辛旻宗/주식프로젝트/key/config.yaml', encoding='UTF-8') as f:
    _cfg = yaml.load(f, Loader=yaml.FullLoader)
APP_KEY = _cfg['APP_KEY']
APP_SECRET = _cfg['APP_SECRET']
ACCESS_TOKEN = ""
CANO = _cfg['CANO']
ACNT_PRDT_CD = _cfg['ACNT_PRDT_CD']
DISCORD_WEBHOOK_URL = _cfg['DISCORD_WEBHOOK_URL']
URL_BASE = _cfg['URL_BASE']

In [ ]:
broker = mojito.KoreaInvestment(
    api_key=APP_KEY, # api kye
    api_secret=APP_SECRET, # secret kye
    acc_no="50101465-01", # 모의계좌 번호
    mock=True # 모의계좌 옵션
)

resp = broker.fetch_ohlcv(
    symbol="005930", # 주식코드 번호
    timeframe='D', # D는 일봉 Y 연봉, 월봉은 기억 안나요
    adj_price=True, # 0.단위 알아서 조정해주는 옵션
    start_day="20240401", # 시작할 날짜인데 해보니까 end_day기준 앞에 100개 가져오는거라 왜 있나 싶어요
    end_day="20240401" # end_day부터 앞에 100개의 데이터를 가져옴
)

pprint.pprint(resp)

In [7]:
df = pd.DataFrame(resp['output2']) # output2에 가격정보가 다 있습니다 output1에는 삼성전자에 대한 내용
dt = pd.to_datetime(df['stck_bsop_date'], format="%Y%m%d")
df.set_index(dt, inplace=True)
df = df[['stck_oprc', 'stck_hgpr', 'stck_lwpr', 'stck_clpr']]
df.columns = ['open', 'high', 'low', 'close']
df.index.name = "date"
df

,open,high,low,close
date,,,,
2022-12-29,56000,56200,55300,55300
2022-12-28,57600,57600,56400,56600
2022-12-27,58000,58400,57900,58100
2022-12-26,58000,58100,57700,57900
2022-12-23,58200,58400,57700,58100
...,...,...,...,...
2022-08-11,59600,60000,59300,59900
2022-08-10,58900,59200,58600,59100
2022-08-09,60600,60700,59600,60000


In [15]:
import mojito
import pprint
from datetime import datetime, timedelta

# 계좌번호 설정
acc_no = "50101465-01"

# 브로커 객체 생성
broker = mojito.KoreaInvestment(
    api_key=APP_KEY,
    api_secret=APP_SECRET,
    acc_no=acc_no
)

# 오늘 날짜 설정
today = datetime.today().strftime("%Y%m%d")

# 2022년 12월 30일부터 시작
start_day = "20221230"
end_day = start_day

# 데이터 리스트
data = []

df = pd.DataFrame(resp['output2'])
dt = pd.to_datetime(df['stck_bsop_date'], format="%Y%m%d")
df.set_index(dt, inplace=True)
df = df[['stck_oprc', 'stck_hgpr', 'stck_lwpr', 'stck_clpr']]
df.columns = ['open', 'high', 'low', 'close']
df.index.name = "date"

# 다음 데이터 추가
while end_day <= today:
    # 100개 일봉 데이터 요청
    resp = broker.fetch_ohlcv(
        symbol="005930",
        timeframe='D',
        adj_price=True,
        end_day=end_day
    )

    # 데이터프레임 생성
    df_temp = pd.DataFrame(resp['output2'])
    dt = pd.to_datetime(df_temp['stck_bsop_date'], format="%Y%m%d")
    df_temp.set_index(dt, inplace=True)
    df_temp = df_temp[['stck_oprc', 'stck_hgpr', 'stck_lwpr', 'stck_clpr']]
    df_temp.columns = ['open', 'high', 'low', 'close']
    df_temp.index.name = "date"

    # 데이터 합치기
    df = pd.concat([df, df_temp])

    # 다음 100일 날짜 설정
    end_day = (datetime.strptime(end_day, "%Y%m%d") + timedelta(days=100)).strftime("%Y%m%d")
    print(end_day)

# 결과 출력
print(df)




20230409
20230718
20231026
20240203
20240513
             open   high    low  close
date                                  
2024-03-22  79600  79900  77800  78900
2024-03-21  79200  79300  77700  79300
2024-03-20  73700  77200  73400  76900
2024-03-19  72300  73000  71700  72800
2024-03-18  72600  73000  72500  72800
...           ...    ...    ...    ...
2023-09-12  70800  71000  70400  70500
2023-09-11  70400  70800  70000  70800
2023-09-08  70200  70300  69600  70300
2023-09-07  70000  70600  69600  70400
2023-09-06  70700  70800  69700  70000

[600 rows x 4 columns]
